# Forecasting Covid-19 Cases with LSTM

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data Preparation

In [ ]:
train_df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test_df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
sub_df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")


In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# pull out state of North Carolina 
nc_train_df=train_df[train_df["Province_State"]=="North Carolina"]
nc_test_df=test_df[test_df["Province_State"]=="North Carolina"]

# pull out state of NY
ny_train_df=train_df[train_df["Province_State"]=="New York"]
ny_test_df=test_df[test_df["Province_State"]=="New York"]


In [ ]:
nc_train_df.shape,ny_train_df.shape

In [ ]:
ny_train_df.head()

In [ ]:
nc_train_df.head()

In [ ]:
# add a data column for days
nc_train_df["days"]=[x for x in range(1,83)]
ny_train_df["days"]=[x for x in range(1,83)]


In [ ]:
nc_train_df.head()

In [ ]:
ny_train_df.head()

In [ ]:
# remake dataframe into 2 columns w/ just our days and cases
nc_train_df=nc_train_df.loc[:,["ConfirmedCases","days"]]
nc_train_df.set_index('days',inplace=True)

ny_train_df=ny_train_df.loc[:,["ConfirmedCases","days"]]
ny_train_df.set_index('days',inplace=True)


In [ ]:
ny_train_df.tail(10)

In [ ]:
plt.figure(figsize = (10, 5))

plt.plot(nc_train_df.index,nc_train_df['ConfirmedCases'], c = 'deeppink',label="North Carolina")
plt.plot(ny_train_df.index,ny_train_df['ConfirmedCases'], c = 'orangered',label="New York")

plt.xlabel("\nDAYS\n", fontsize=10)
plt.ylabel("\nCASES\n", fontsize=10)
plt.title("Covid-19 Cases \n", fontsize=18)
plt.legend()
plt.show()

In [ ]:
# Let's load the required libs.
# We'll be using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

### Normalize data

In [ ]:

def normalize_data(train_df):
    # get data values from the pandas data frame.
    values = train_df.values.astype("float32")
    # apply the MinMax scaler from sklearn
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(values)
    # save normalized dataset into df
    series = pd.DataFrame(dataset)
    
    return dataset,series;


In [ ]:
dataset_nc,series_nc=normalize_data(nc_train_df)
dataset_ny,series_ny=normalize_data(ny_train_df)

In [ ]:
def prepare_data(dataset,TRAIN_SIZE=0.90):
    #dataset= shuffle(dataset)

    # Shuffle training data.
    train_size = int(len(dataset) * TRAIN_SIZE)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    print("(training set, test set): " + str((len(train), len(test))))

    return train,test;

train_nc,test_nc=prepare_data(dataset_nc,0.90)
train_ny,test_ny=prepare_data(dataset_ny,0.90)

In [ ]:

def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [ ]:
# Create test and training sets for one-step-ahead regression.
def make_one_step_regr_sets(train, test, window_size=1):
    window_size = 1
    train_X, train_Y = create_dataset(train, window_size)
    test_X, test_Y = create_dataset(test, window_size)
    print("Original training data shape:")
    print(train_X.shape)

    # Reshape the input data into appropriate form for Keras.
    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
    test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
    print("New training data shape:")
    print(train_X.shape)
    return train_X, train_Y, test_X, test_Y;



In [ ]:
nc_train_X, nc_train_Y, nc_test_X, nc_test_Y=make_one_step_regr_sets(train_nc, test_nc, window_size=1)

ny_train_X, ny_train_Y, ny_test_X, ny_test_Y=make_one_step_regr_sets(train_ny, test_ny, window_size=1)


### Simple LSTM Network

* 1 input layer  
* 1 LSTM layer w/ 4 blocks  
* 1 Dense layer to produce a single output
* MSE loss function

In [ ]:
def fit_model(train_X, train_Y, window_size = 1):
    model = Sequential()
    
    model.add(LSTM(4, 
                   input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    model.fit(train_X, 
              train_Y, 
              epochs = 100, 
              batch_size = 1)
    model.summary()
    return(model)



In [ ]:
# Fit the first model.
nc_model = fit_model(nc_train_X, nc_train_Y, window_size)

In [ ]:
# Fit the first model.
ny_model = fit_model(ny_train_X, ny_train_Y, window_size)

In [ ]:
model.summary()

In [ ]:
import math

def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred, orig_data)




In [ ]:
nc_rmse_train, nc_train_predict,nc_train_orig = predict_and_score(nc_model, nc_train_X,nc_train_Y)
nc_rmse_test, nc_test_predict, nc_test_orig = predict_and_score(nc_model, nc_test_X, nc_test_Y)

print("Training data score: %.2f RMSE" % nc_rmse_train)
print("Test data score: %.2f RMSE" % nc_rmse_test)

In [ ]:
ny_rmse_train, ny_train_predict,ny_train_orig = predict_and_score(ny_model, ny_train_X,ny_train_Y)
ny_rmse_test, ny_test_predict, ny_test_orig = predict_and_score(ny_model, ny_test_X, ny_test_Y)

print("Training data score: %.2f RMSE" % ny_rmse_train)
print("Test data score: %.2f RMSE" % ny_rmse_test)

In [ ]:
def plot_predictions(dataset, train_predict, test_predict,title="Covid-19 Cases"):
    # Start with training predictions.
    train_predict_plot = np.empty_like(dataset)
    train_predict_plot[:, :] = np.nan
    train_predict_plot[window_size:len(train_predict) + window_size, :] = train_predict

    # Add test predictions.
    test_predict_plot = np.empty_like(dataset)
    test_predict_plot[:, :] = np.nan
    test_predict_plot[len(train_predict) + (window_size * 2) + 1:len(dataset) - 1, :] = test_predict

    sns.set_context('poster')
    # Create the plot.
    plt.figure(figsize = (15, 5))
    plt.plot(scaler.inverse_transform(dataset), label = "True value", color='dimgrey')
    plt.plot(train_predict_plot, label = "Training set prediction",color='mediumpurple')
    plt.plot(test_predict_plot, label = "Test set prediction",color='chartreuse')
    plt.xlabel("Days")
    plt.ylabel("Number of Cases")
    plt.title("{} \n".format(title))
    plt.legend()
    plt.show()

In [ ]:
plot_predictions(dataset_nc, nc_train_predict, nc_test_predict,title="NC Covid-19 Cases")
plot_predictions(dataset_ny, ny_train_predict, ny_test_predict,title="NY Covid-19 Cases")

In [ ]:
#print(model.history.history.keys())
nc_losses_lstm =nc_model.history.history['loss']
ny_losses_lstm =ny_model.history.history['loss']

plt.figure(figsize=(12,4))
plt.xticks(np.arange(0,120,10))
plt.plot(range(len(losses_lstm)),nc_losses_lstm, c="deeppink", label="nc")
plt.plot(range(len(losses_lstm)),ny_losses_lstm, c="orangered", label="ny")
plt.title("Loss\n")
plt.legend();


### Bidirectional Model

In [ ]:
opt = Adam(lr=0.001)
# define model
bi_model = Sequential()
bi_model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 1)))
bi_model.add(Dense(1))
bi_model.compile(optimizer=opt, loss='mse')
bi_model.summary()

### Fit

In [ ]:
nc_bi_model_history = bi_model.fit(nc_train_X, nc_train_Y, epochs=600, batch_size=256, validation_data=(nc_test_X, nc_test_Y),
                    verbose=0, shuffle=False,callbacks=[EarlyStopping(patience=10)])

In [ ]:
ny_bi_model_history = bi_model.fit(ny_train_X, ny_train_Y, epochs=600, batch_size=256, validation_data=(ny_test_X, ny_test_Y),
                    verbose=0, shuffle=False,callbacks=[EarlyStopping(patience=10)])

### Predict

In [ ]:
def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return score, pred;



In [ ]:
ny_bi_rmse_train, ny_bi_train_predict = predict_and_score(bi_model, ny_train_X, ny_train_Y)
ny_bi_rmse_test, ny_bi_test_predict = predict_and_score(bi_model, ny_test_X, ny_test_Y)
nc_bi_rmse_train, nc_bi_train_predict = predict_and_score(bi_model, nc_train_X, nc_train_Y)
nc_bi_rmse_test, nc_bi_test_predict = predict_and_score(bi_model, nc_test_X, nc_test_Y)

print('NY Predictions:')
print("Training data score: %.2f RMSE" % ny_bi_rmse_train)
print("Test data score: %.2f RMSE" % ny_bi_rmse_test)
print('\nNC Predictions:')

print("Training data score: %.2f RMSE" % nc_bi_rmse_train)
print("Test data score: %.2f RMSE" % nc_bi_rmse_test)

In [ ]:
plot_predictions(dataset_ny, ny_bi_train_predict, ny_bi_test_predict,title="NY Covid-19 Cases \nbidirectional LSTM")
plot_predictions(dataset_nc, nc_bi_train_predict, nc_bi_test_predict,title="NC Covid-19 Cases \nbidirectional LSTM")

In [ ]:
#print(model.history.history.keys())
nc_losses_lstm =nc_model.history.history['loss']
ny_losses_lstm =ny_model.history.history['loss']

plt.figure(figsize=(12,4))
plt.xticks(np.arange(0,120,10))
plt.plot(range(len(losses_lstm)),nc_losses_lstm, c="deeppink", label="nc")
plt.plot(range(len(losses_lstm)),ny_losses_lstm, c="orangered", label="ny")
plt.title("Bidirectional LSTM Loss\n")
plt.legend();


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout, Bidirectional, GRU
from keras.optimizers import SGD, Adadelta, RMSprop, Adam, Nadam
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping
from tqdm import tqdm


reg = l1_l2(l1=0.0015, l2=0.0)
opt = Adam(lr=0.0015)

bi_model = Sequential()
bi_model.add(Bidirectional(LSTM(140, activation='relu', return_sequences=True, kernel_regularizer=reg, recurrent_regularizer=reg)))
bi_model.add(Bidirectional(LSTM(140, activation='relu', return_sequences=True, kernel_regularizer=reg, recurrent_regularizer=reg)))
bi_model.add(Bidirectional(LSTM(140, activation='relu', kernel_regularizer=reg, recurrent_regularizer=reg)))
bi_model.add(Dense(28))
bi_model.add(Dense(1))
bi_model.add(Activation('linear'))
bi_model.compile(loss='mean_absolute_error', optimizer=opt)

bi_model_history = bi_model.fit(nc_train_X, nc_train_Y, epochs=600, batch_size=256, validation_data=(test_X, test_Y),
                    verbose=0, shuffle=False,callbacks=[EarlyStopping(patience=10)])

https://www.kaggle.com/ternaryrealm/lstm-time-series-explorations-with-keras